# Principles of Digital Communication

## To do : 
- Avec le simple repetition code (pas de parity check), avec un message de 10 charactères, déterminer le nombre de répétitions à partir duquel sur 100 runs, aucune erreur n'est obtenue. Super important : c'est la baseline qu'on va utiliser comme nombre pour parity_incrust sur le second point.
- Avec le code hybride : jouer avec les paramètres k, block_length, parity_incrust, batch_size ( avec batch_size == block_length), pour déterminer les paramètres optimaux pour que sur 100 runs, il n'y ait pas d'erreurs. Je doute que ce nombre vaille 0, mais bon, c'est de l'optimisation, c'est vous les data scientists :) . 
- En réalité, le code "hybride" est une variante du LDPC (une méthode d'encodage qui est utilisée dans le standard de la 5G) que j'ai essayé d'implémenter, mais qui n'est pas optimale. Pour faire simple, mon code hybride utilise 1 parity bit par set disjoint de 8 bits d'information, l'optimisation du LDPC consiste à rajouter des bits de protection sur des set non-disjoints. Par exemple : imaginez que vous avez 24 bits d'information à protéger, le code hybride va avoir un bit de parity après le 8ème bit, un bit de parity après le 16ème bit, et un bit de parity après le 24ème bit. Le LDPC va rajouter un bit de parity qui couvre les bits 1 à 8, puis un bit de parity qui couvre les bits 7 à 14, puis un bit de parity qui couvre les bits 12 à 19, puis un bit de parity qui couvre les bits 17 à 24. En gros, les bits de parity couvrent des "overlapping sets", mais faut en rajouter. Je soupçonne que si c'est bien implémenté, on peut atteindre un rate beaucoup plus bas, en ayant besoin de moins de répétitions, faut l'implémenter.
- Explorer ce qu'il est possible de faire avec d'autres types de codes : si vous pouvez explorer les "Tornado Codes" (qui sont , "Reed-Solomon codes", "Turbo Codes" et "Reed-Muller Codes", ce serait top! En fait, il faut savoir que le code sur lequel on a travaillé dans le homework 2 (le Hadamard Code), est en fait un cas particulier d'un Reed-Muller code. Vous pouvez partir de là.

### Imports

In [175]:
import helpers
import numpy as np



### Encoding 

In [176]:
input = "test.txt"
output = "output.txt"


In [177]:
#returns a list of characters from a given textfile
text = helpers.read_text_file(input)

## Hybrid Encoding

In [178]:
#NE PAS TOUCHER, WALID BOSSE ACTUELLEMENT SUR CA
#Ce bloc là fonctionne bien, il incruste bien les parity bits pour un block de taille block_length


text_modified_binary_original = helpers.chars_to_bits_partial(text)
text_for_compare = helpers.from_zero_to_neg(text_modified_binary_original)
block_length = 8   
temp = []
def parity_incrust(l):
    res = []
    n = len(l)
    for i in range(int(n/block_length)):
        acc_1 = 0
        acc_3 = 0
        
        for j in range(block_length):
            # First parity bit
                
            if(j==4 or j==0 or j==2  or j==5 or j == 6 or j == 7 ):
                acc_1 += l[i*block_length+j]

                
            if(j==0 or j==1 or j==3  or j==5 or j == 6 or j == 7 ):
                acc_3 += l[i*block_length+j]
            
            
        acc_1 = acc_1%2
        acc_3 = acc_3%2
        
        res.extend(l[i*block_length:(i+1)*block_length])
        res.append(acc_1)
        temp.append(acc_1)
        res.append(acc_3)
        temp.append(acc_3)
    #print(parity_bits_for_print)
    return res

            
text_modified_binary = parity_incrust(text_modified_binary_original)
text_modified_binary = list(map(int, text_modified_binary))



In [179]:
len(text_modified_binary_original)

640

# Hybrid Repetition

In [180]:
#faut jouer avec k et parity_redundancy pour trouver un bon équilibre

k = 52
batch_size = 8
parity_redundancy = 111

In [181]:
print(k*640 + ((2*640)/batch_size)*parity_redundancy)
assert(k*640 + ((2*640)/batch_size)*parity_redundancy < 51200)


51040.0


In [182]:
text_repeated = helpers.repetition_hybrid(text_modified_binary,k,batch_size,parity_redundancy)
text_repeated = helpers.from_zero_to_neg(text_repeated)


In [183]:
len(text_repeated)

51040

In [184]:
np.savetxt("input.txt",text_repeated,fmt='%i')

### Sending to server

In [186]:
%run client.py --input_file="input.txt" --output_file="output.txt" --srv_hostname=iscsrv72.epfl.ch --srv_port=80

# Decoding

In [187]:
output = open('output.txt', 'r').read().splitlines()

In [188]:
# convert output from char to float 
output = [float(i) for i in output]

In [189]:
print(len(output))

51040


## Hybrid decoding

In [190]:
#NE PAS TOUCHER CETTE SECTION, WALID BOSSE ACTUELLEMENT SUR CA


#Works on the reconstruction and helper_reconstruct part
#Cette méthode prend le fichier reçu et renvoie 3 listes : 
# reconstruction : Les bits du message reconstruits en moyennant leur répétitions clippés à -1;+1
# helper_reconstruct : les "moyennes" des répétitions des bits bruités
# parity_list : la même chose que répétition, mais pour les bits de parité


def hybrid_decode_builder(encoded):
    reconstruction = []
    helper_reconstruct=[]
    parity_list = []
    full_length = len(encoded)
    full_batch_size = k * batch_size + 2*parity_redundancy
    num_batches = int(full_length/full_batch_size)
    #assert(num_batches==80)
    #assert(full_batch_size==570)
    # NUM BATCHES 80
    for batch in range(num_batches):
        # BATCH SIZE 8
        for i in range(batch_size):
            acc = 0
            for j in range(k):
                index = batch * full_batch_size + i*k + j
                acc += encoded[index]
                #print( "acc : ", acc, " , j = ", j, " i = ", i)
            helper_reconstruct.append(acc)
            if acc > 0 : 
                reconstruction.append(1)
            else : 
                reconstruction.append(-1)
            acc = 0
            

        acc_3 = 0
        for j in range(parity_redundancy):
            index = batch * full_batch_size + batch_size*k+ j
            index_3 = index + parity_redundancy
            acc += encoded[index]
            acc_3 += encoded[index_3]
            
            
        if acc > 0 : 
            parity_list.append(1)
        else : 
            parity_list.append(-1)
        if acc_3 > 0 : 
            parity_list.append(1)
        else : 
            parity_list.append(-1)
            
        
    return reconstruction,helper_reconstruct,parity_list



    

In [191]:
def parity_checking(rec,helper,parity_l):
    counter = 0
    n = len(parity_l)
    bs = int(len(rec)/n)*2
    assert(bs==batch_size)
    result = []
    result.extend(rec)
    bs = batch_size
    for i in range(0,n,2):
        parity_bit_noisy = parity_l[i]

        parity_bit_noisy_3 = parity_l[i+1]
        
        half_i = int(i/2)
        
        parity_bit_computed_3 = helpers.parity_find_3(result[half_i*bs:(half_i+1)*bs])       
 
        if parity_bit_noisy_3!=parity_bit_computed_3 :
            counter += 1 
            print('corrected_one')
            
            j = helpers.find_absolute_min_index(helper[half_i*bs:(half_i+1)*bs])
            print('error at index ', j+half_i*bs ,'--  change ' , result[j+half_i*bs] , ' to ' , -result[j+half_i*bs] ) 
            result[j+half_i*bs] = -result[j+half_i*bs]
        
            parity_bit_computed = helpers.parity_find(result[half_i*bs:(half_i+1)*bs])
            if parity_bit_noisy!=parity_bit_computed :
                counter +=1
                print('corrected_two')
                j = helpers.find_absolute_min_index_2(helper[half_i*bs:(half_i+1)*bs])
                print('error at index ', j+half_i*bs ,'--  change ' , result[j+half_i*bs] , ' to ' , -result[j+half_i*bs] ) 

                result[j+half_i*bs] = -result[j+half_i*bs]
                continue
                
                    

                                
                
            
        
        parity_bit_computed = helpers.parity_find(result[half_i*bs:(half_i+1)*bs])
        if parity_bit_noisy!=parity_bit_computed :
            counter +=1
            print('corrected_two')
            j = helpers.find_absolute_min_index(helper[half_i*bs:(half_i+1)*bs])
            print('error at index ', j+half_i*bs ,'--  change ' , result[j+half_i*bs] , ' to ' , -result[j+half_i*bs] ) 

            result[j+half_i*bs] = -result[j+half_i*bs]
            

                
                
                
    print("number of errors detected : ",counter)
            
        
        
            
    return result



In [192]:
# parse input 
reconstruction,helper_reconstruct,parity_list = hybrid_decode_builder(output)


In [193]:
len(parity_list)

160

In [194]:
text_for_compare[136:144]

[-1, 1, 1, 1, -1, -1, 1, 1]

In [195]:
reconstruction[136:144]

[-1, 1, 1, 1, -1, -1, 1, 1]

In [196]:
helper_reconstruct[136:144]

[-107.05309945420898,
 79.67336657957699,
 71.55835491896538,
 67.8897774583431,
 -31.716977818763812,
 -57.241384926757895,
 61.408376928850956,
 38.658200014169445]

In [197]:
# correct 
reconstruction = parity_checking(reconstruction,helper_reconstruct,parity_list)


corrected_one
error at index  63 --  change  1  to  -1
corrected_one
error at index  77 --  change  1  to  -1
corrected_one
error at index  112 --  change  1  to  -1
corrected_one
error at index  128 --  change  1  to  -1
corrected_two
error at index  314 --  change  -1  to  1
corrected_one
error at index  413 --  change  -1  to  1
corrected_one
error at index  536 --  change  1  to  -1
corrected_one
error at index  568 --  change  1  to  -1
corrected_two
error at index  583 --  change  -1  to  1
corrected_one
error at index  584 --  change  -1  to  1
number of errors detected :  10


In [198]:
reconstruction[246:248]

[-1, -1]

In [199]:
len(parity_list)

160

## Make sure our parity bits contain no errors

In [200]:
helpers.compute_Hamming(helpers.from_zero_to_neg(temp),parity_list)

(0, [])

### Hybrid Error Evaluation

In [201]:
v1,v2 = helpers.compute_Hamming(text_for_compare,reconstruction)
print(v1)
print(v2)


4
[(578, 1), (583, -1), (584, -1), (589, -1)]


In [171]:
helper_reconstruct[72:80]

[-50.90485622154812,
 84.9174302554785,
 31.5528572663738,
 -71.54468483138272,
 15.075118796483059,
 -13.240698012468783,
 -45.95050990905744,
 53.8034941768679]

In [172]:
text_for_compare[30]

-1

In [173]:
reconstruction[30]

-1

In [174]:
r = helpers.bits_to_chars(reconstruction)
r2 = helpers.from_bits_to_chars(r)
print(str(r2))

ayaài w wmlid amls poõr la vie ayadi r walid amis pour la vie aqadi w walid123 



### Classic Error Evaluation

In [101]:
#v1 = helpers.compute_Hamming(text_modified_binary,reconstruction)
#v3 = helpers.compute_Hamming(text_modified_binary,reconstruction3)
#print(v1)
#print(v3)
#text_modified_binary==reconstruction

## Write on file the received value

In [102]:
write_chars_on_file(res2)

NameError: name 'write_chars_on_file' is not defined

# Calculating noise statistics

partir d'une liste -1, 1.
1- calculer le mean de ma liste de départ 
2- 

In [91]:
k = 50000

In [92]:
input_list = helpers.generate_random_list(-1, k)

In [ ]:
mean_transmitted = np.mean(input_list)
std_transmitted = np.std(input_list)
print(mean_transmitted)
print(std_transmitted)

### Sending to server

In [ ]:
np.savetxt("input.txt",input_list,fmt='%i')

In [ ]:
%run client.py --input_file="input.txt" --output_file="output25.txt" --srv_hostname=iscsrv72.epfl.ch --srv_port=80

### Decoding

In [ ]:
# convert output from char to float 
output1 = open('output.txt', 'r').read().splitlines()
output1 = [float(i) for i in output]

In [ ]:
output2 = open('output2.txt', 'r').read().splitlines()
output2 = [float(i) for i in output2]

In [ ]:
output3 = open('output3.txt', 'r').read().splitlines()
output3 = [float(i) for i in output3]

In [ ]:
output4 = open('output4.txt', 'r').read().splitlines()
output4 = [float(i) for i in output4]

In [ ]:
output5 = open('output5.txt', 'r').read().splitlines()
output5 = [float(i) for i in output5]

In [ ]:
output6 = open('output6.txt', 'r').read().splitlines()
output6 = [float(i) for i in output6]

In [ ]:
output7 = open('output7.txt', 'r').read().splitlines()
output7 = [float(i) for i in output7]


In [ ]:
output8 = open('output8.txt', 'r').read().splitlines()
output8 = [float(i) for i in output8]


In [ ]:
output9 = open('output9.txt', 'r').read().splitlines()
output9 = [float(i) for i in output9]


In [ ]:
output10 = open('output10.txt', 'r').read().splitlines()
output10 = [float(i) for i in output10]


In [ ]:
output11 = open('output11.txt', 'r').read().splitlines()
output11 = [float(i) for i in output11]


In [ ]:
output12 = open('output12.txt', 'r').read().splitlines()
output12 = [float(i) for i in output12]


In [ ]:
output13 = open('output13.txt', 'r').read().splitlines()
output13 = [float(i) for i in output13]


In [ ]:
output14 = open('output14.txt', 'r').read().splitlines()
output14 = [float(i) for i in output14]


In [ ]:
output15 = open('output15.txt', 'r').read().splitlines()
output15 = [float(i) for i in output15]


In [ ]:
output16 = open('output16.txt', 'r').read().splitlines()
output16 = [float(i) for i in output16]


In [ ]:
output17 = open('output17.txt', 'r').read().splitlines()
output17 = [float(i) for i in output17]


In [ ]:
output18 = open('output18.txt', 'r').read().splitlines()
output18 = [float(i) for i in output18]


In [ ]:
output19 = open('output19.txt', 'r').read().splitlines()
output19 = [float(i) for i in output19]


In [ ]:
output20 = open('output20.txt', 'r').read().splitlines()
output20 = [float(i) for i in output20]


In [ ]:
output21 = open('output21.txt', 'r').read().splitlines()
output21 = [float(i) for i in output21]

In [ ]:
output22 = open('output22.txt', 'r').read().splitlines()
output22 = [float(i) for i in output22]

In [ ]:
output23 = open('output23.txt', 'r').read().splitlines()
output23 = [float(i) for i in output23]

In [ ]:
output24 = open('output24.txt', 'r').read().splitlines()
output24 = [float(i) for i in output24]

In [ ]:
output25 = open('output25.txt', 'r').read().splitlines()
output25 = [float(i) for i in output25]

In [ ]:
big_output = output1#+output2+output3+output4+output5+output6+output7+output8+output9+output10+output11+output12+output13+output14+output15+output16+output17+output18+output19+output20+output21+output22+output23+output24+output25


In [ ]:
mean_received = np.mean(big_output)
std_received = np.std(big_output)
print(mean_received)
print(std_received)
print(std_received**2)

shift = 1 + mean_received
print(shift)

In [ ]:
channel_capacity1 = np.log2(1+ (1/(std_received**2)))/2
print(channel_capacity1)
channel_capacity2 = np.log2(1+ np.sqrt(2/(np.pi * np.e)))
print(channel_capacity2)

minimum_n = 640/channel_capacity1



The best possible reliable code must send at least about 11000 bits.

In [ ]:
mean_received = np.mean(output)
std_received = np.std(output)
print(mean_received)
print(std_received)

In [ ]:
noise_mean = mean_received - mean_transmitted
noise_std = std_received - std_transmitted
print('noise_mean: ', noise_mean)
print('noise_std: ', noise_std)

Pour -1 : mean = 0.991298758983935 , std = 3.552037915973558

Pour 1 : mean = -0.9995436520014286 , std = 3.5203711872123264

Pour 1, -1 : mean = 0.011907669713883446, std = 2.546875294224225

# Param estimation

In [24]:
import collections
import time
dict = collections.defaultdict(list)
for k in range(40,80):
    for parity_redundancy in range(100,200,10):
        for i in range(10):
            print(k)
            text_modified_binary_original = helpers.chars_to_bits_partial(text)
            text_for_compare = helpers.from_zero_to_neg(text_modified_binary_original)
            text_modified_binary = parity_incrust(text_modified_binary_original)
            text_modified_binary = list(map(int, text_modified_binary))

            batch_size = 8
            text_repeated = helpers.repetition_hybrid(text_modified_binary,k,batch_size,parity_redundancy)
            text_repeated = helpers.from_zero_to_neg(text_repeated)
            np.savetxt("input.txt",text_repeated,fmt='%i')

            time.sleep(30)
            %run client.py --input_file="input.txt" --output_file="output.txt" --srv_hostname=iscsrv72.epfl.ch --srv_port=80

            output = open('output.txt', 'r').read().splitlines()
            # convert output from char to float 
            output = [float(i) for i in output]
            reconstruction,helper_reconstruct,parity_list = hybrid_decode_builder(output)
            reconstruction = parity_checking(reconstruction,helper_reconstruct,parity_list)
            print(len(reconstruction),len(text_for_compare))
            v1,v2 = helpers.compute_Hamming(text_for_compare,reconstruction)
            dict[(k,batch_size,parity_redundancy)].append(v1)
    print(k)

dict_1 = {k: sum(v)/len(v) for k, v in dict.items()}
f = open("dict.txt","w")
f.write( str(dict) )
f.close()
f = open("dict_1.txt","w")
f.write( str(dict_1) )
f.close()

40
[1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1]
848 640


AssertionError: 